In [1]:
#importing libraries
import pandas as pd
import numpy as np
from google.cloud import bigquery, bigquery_storage
from utils import *
from features import *
import warnings 
warnings.filterwarnings('ignore')

import random
random.seed(12345)

In [2]:
## query
#GA, Content and Sentiment
query = """
    WITH ga_data AS (
        SELECT
            GA_fullVisitorId,
            GA_visitStartTime, 
            GA_date,
            GA_pagePath,
            GA_dfpNewZone,
            GA_visitNumber,
            GA_pageViews,
            GA_scrollDepth,
            timeOnPage,  
            GA_cmsNaturalId,
            GA_deviceOperatingSystem,
            GA_deviceCategory,
            GA_deviceBrowser,
            GA_country,
            GA_referralGroup,
            GA_primaryChannel,
            GA_primarySection
        FROM
            `api-project-901373404215.DataMart.v_DataMart_updated`
        WHERE
            ga_date BETWEEN '2022-02-01' and '2022-04-30'
            
    ),
    sentiment as
    (
SELECT * EXCEPT (rank)
    FROM
    (
    SELECT 
          lower(natid) as sentiment_natid,
          pub_date, 
          clean_body, 
          sentiment_score,
          RANK() OVER (PARTITION BY natid ORDER BY pub_date DESC) AS rank
    FROM 
      `api-project-901373404215.sentiment.article_sentiment`
    )
    WHERE
           rank = 1 ),
    content_t AS (
        SELECT
            content_t.natid AS content_natid,
            publish_date,
            title,
            body,
            iab_cat_t.tier1
        FROM (
            SELECT 
                natid,
                EXTRACT(DATE FROM date_et) AS publish_date,
                title,
                body
            FROM (
                SELECT DISTINCT
                DATETIME(date, "America/New_York") as date_et,
                title,
                body,
                LOWER(NaturalId) AS natid,
                RANK() OVER (PARTITION BY naturalid ORDER BY timestamp DESC) AS mostrecent
            FROM
                `api-project-901373404215.Content.content`
            WHERE
                Visible is true
                    AND type in (
                    'blog',
                    'blogslide',
                    'magazine'
                )
            )
        WHERE 
            mostrecent = 1
        ) AS content_t
        INNER JOIN (
            SELECT
                * EXCEPT (rank)
            FROM (
                SELECT DISTINCT
                    category AS tier1,
                    natid,
                    RANK() OVER (PARTITION BY natid ORDER BY ts DESC) AS rank
                FROM
                    `api-project-901373404215.DataMart.iab_cat_v2` 
            )
        WHERE
            rank = 1
        ) AS iab_cat_t
    ON 
        LOWER(iab_cat_t.natid) = LOWER(content_t.natid)
    ),
ga_content as 
    (
    SELECT
        GA_date,
        GA_fullVisitorId, 
        GA_visitStartTime,
        GA_visitNumber,
        GA_cmsNaturalId,
        publish_date,
        title,
        body,
        tier1,
        GA_pagePath,
        GA_dfpNewZone,
        timeOnPage,
        GA_scrollDepth,
        GA_pageViews,
        GA_deviceOperatingSystem,
        GA_deviceCategory,
        GA_deviceBrowser,
        GA_country,
        GA_referralGroup,
        GA_primaryChannel,
        GA_primarySection
    FROM
        ga_data
    
    LEFT OUTER JOIN
        content_t
    ON 
        LOWER(GA_cmsNaturalId) = LOWER(content_natid)
    WHERE
        GA_fullVisitorId IN (
          SELECT DISTINCT 
              GA_fullVisitorId 
          FROM 
              `api-project-901373404215.lookalike_data.zoominfo_labels`)
)

SELECT 
    GA_date,
        GA_fullVisitorId, 
        GA_visitStartTime,
        GA_visitNumber,
        GA_cmsNaturalId,
        publish_date,
        title,
        body,
        clean_body,
        sentiment_score,
        tier1,
        GA_pagePath,
        GA_dfpNewZone,
        timeOnPage,
        GA_scrollDepth,
        GA_pageViews,
        GA_deviceOperatingSystem,
        GA_deviceCategory,
        GA_deviceBrowser,
        GA_country,
        GA_referralGroup,
        GA_primaryChannel,
        GA_primarySection
    FROM
        ga_content
    
    LEFT OUTER JOIN
        sentiment
    ON 
        LOWER(GA_cmsNaturalId) = LOWER(sentiment_natid)
    """
#subscriber data
query_sub= """
    WITH 
    active_subs AS (
        SELECT DISTINCT
                user_id_uid
        FROM
            `api-project-901373404215.piano.subscriber_details`
        WHERE
            # Filter for the 'universal' subscriptions only
            resource_id_rid IN UNNEST(['RKPEVDB', 'R8W03AS'])
            AND status='active'
            AND total__refunded<1
    ),
    -- Join with the datamart to get the ga fullvid
    sub_fullvids AS (
        SELECT DISTINCT
            ga_fullvisitorid
        FROM
            active_subs
        INNER JOIN
            `api-project-901373404215.DataMart.v_DataMart_updated`
        ON
            LOWER(ga_pianoId) = LOWER(user_id_uid)
    )
    -- Create a flag for subscribers based off the ga fullvid
    SELECT
        *
    FROM

        sub_fullvids
    """

In [3]:
## Fetching data

bqclient = bigquery.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

#GA_Content_Sentiment
df = bqclient.query(query).result().to_dataframe(bqstorage_client=bqstorageclient)
print(df.shape)

#Subscriber data

sub = bqclient.query(query_sub).to_dataframe()
print(sub.shape)

(1196150, 23)
(207637, 1)


In [4]:
len(df.GA_fullVisitorId.unique())

123430

In [5]:
#Newsletter Unsubscription

nl = df.loc[df['GA_pagePath'].str.contains("newsletter/unsubscribe-confirmation", case=False)]
print(nl.shape)
nl_unsub=nl.GA_fullVisitorId.unique().tolist()
print(len(nl_unsub))

(6101, 23)
5216


In [6]:
#creating a subscribers list
sub_list=sub.ga_fullvisitorid.values.tolist()
len(sub_list)

207637

In [ ]:
# Upload to BQ
DATASET_ID = 'lookalike_trail_data'
TABLE_ID = 'sample_prediction_data_feb_april_training_raw'

upload_bq(
    dataset_id=DATASET_ID,
    table_id=TABLE_ID,
    df=df,
    write_truncate=True # Will overwrite current table
)

In [8]:
# Transform the user features
df_trans = get_user_feats(df)
df_trans.shape

(123430, 189)

In [9]:
#Checking NA values
df_trans.isna().sum()

GA_fullVisitorId      0
session_pvs_mean      0
session_pvs_median    0
session_top_mean      0
session_top_median    0
                     ..
day_of_mon_27_top     0
day_of_mon_28_top     0
day_of_mon_29_top     0
day_of_mon_30_top     0
day_of_mon_31_top     0
Length: 189, dtype: int64

In [10]:
#create a new feature "Newsletter subscriber"
df_trans['nl_subscription']=np.where(df_trans['GA_fullVisitorId'].isin (nl_unsub), 'nl_unsubscriber', 'nl_subscriber')

In [11]:
#create a new feature "subscriber"
df_trans['subscriber']=np.where(df_trans['GA_fullVisitorId'].isin (sub_list), 'subscriber', 'non_subscriber')

In [12]:
df_trans.head()

,GA_fullVisitorId,session_pvs_mean,session_pvs_median,session_top_mean,session_top_median,sentiment_score_mean,sentiment_score_median,rf_content_aggregators,rf_direct,rf_fbia,...,day_of_mon_24_top,day_of_mon_25_top,day_of_mon_26_top,day_of_mon_27_top,day_of_mon_28_top,day_of_mon_29_top,day_of_mon_30_top,day_of_mon_31_top,nl_subscription,subscriber
0,1000256730503871906,1.925926,1.0,133.010626,21.0,-0.026689,0.000000,0.0,0.0,0.0,...,53.0,9.0,0.0,0.0,1822.0,559.666667,1.0,0.0,nl_subscriber,subscriber
1,10002623166896223159,1.000000,1.0,3.000000,3.0,-0.050950,-0.050950,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,nl_subscriber,non_subscriber
2,10002832795477719003,1.000000,1.0,124.000000,44.5,0.341238,0.568571,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,nl_subscriber,non_subscriber
3,1000292907016284907,1.000000,1.0,7.000000,7.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,nl_subscriber,subscriber
4,1000334800127973090,1.000000,1.0,5.000000,5.0,-0.040309,-0.040309,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,nl_subscriber,non_subscriber


In [12]:
# # ## Create data set
# from google.cloud import bigquery

# DATASET_ID = '{}.lookalike_trail_data'

# # # Construct a BigQuery client object.
# client = bigquery.Client()

# # # TODO(developer): Set dataset_id to the ID of the dataset to create.
# dataset_id = DATASET_ID.format(client.project)

# # # Construct a full Dataset object to send to the API.
# dataset = bigquery.Dataset(dataset_id)

# # # TODO(developer): Specify the geographic location where the dataset should reside.
# dataset.location = "US"

# # # Send the dataset to the API for creation, with an explicit timeout.
# # # Raises google.api_core.exceptions.Conflict if the Dataset already
# # # exists within the project.
# dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
# print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset api-project-901373404215.lookalike_trail_data


In [13]:
# Upload to BQ
DATASET_ID = 'lookalike_trail_data'
TABLE_ID = 'zoominfo_user_feats_feb_april_training_v1' #change the name if there is any change in training dates

upload_bq(
    dataset_id=DATASET_ID,
    table_id=TABLE_ID,
    df=df_trans,
    write_truncate=True # Will overwrite current table
)

zoominfo_user_feats_feb_april_training_v1 loaded into BQ
